In [10]:
from utils import *

In [14]:
R=5
k=10
L=10
path='dataset/DB1K.csv'
DBGraph=Initialize_Random_Graph(path,R)

medoid,min_distance=get_medoid(DBGraph)
#how to choose query? how to choose k,L?
rng = np.random.default_rng(50)
Query=rng.random((1, 70), dtype=np.float32)
Greedy_Search(DBGraph,Query,k,L)

# for i,vertex in enumerate(DBGraph):
#             print(vertex)
#             if(i==3):
#                 break

TypeError: unsupported operand type(s) for -: 'set' and 'float'